In [1]:
import pandas as pd

df_airbnb = pd.read_csv("AB_NYC_2019.csv")

In [2]:
df_airbnb.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [3]:
df_airbnb.shape

(48895, 16)

In [4]:
df_airbnb.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')

### Convertir el dataset de Airbnb en varias tablas:

1. hosts
- host_id (PK)
- host_name
- calculated_host_listings_count

2. neighbourhoods
- neighbourhood_id (PK, AUTO_INCREMENT)
- neighbourhood_group
- neighbourhood

3. listings
- listing_id (PK)
- name
- host_id (FK: hosts.host_id)
- neighbourhood_id (FK: neighbourhoods.neighbourhood_id)
- room_type
- price
- minimum_nights
- availability_365

4. reviews
- review_id (PK, AUTO_INCREMENT)
- listing_id (FK: listings.listing_id)
- number_of_reviews
- last_review (DATE)
- reviews_per_month

### Tabla Host

In [5]:
hosts = (
    df_airbnb[
        ['host_id', 'host_name', 'calculated_host_listings_count']
    ]
    .drop_duplicates()
    .reset_index(drop=True)
)

# Guardamos el csv
hosts.to_csv('hosts.csv', index=False)

In [6]:
df_hosts = pd.read_csv("hosts.csv")
df_hosts.head()

,host_id,host_name,calculated_host_listings_count
0,2787,John,6
1,2845,Jennifer,2
2,4632,Elisabeth,1
3,4869,LisaRoxanne,1
4,7192,Laura,1


In [7]:
# Ver duplicados en host_id (PK)
print("Duplicados en host_id:", df_hosts['host_id'].duplicated().sum())

# Nulos
print(hosts.isnull().sum())

Duplicados en host_id: 0
host_id                            0
host_name                         18
calculated_host_listings_count     0
dtype: int64


### Tabla Neighbourhoods

In [ ]:
nb = (
    df_airbnb[
        ['neighbourhood_group', 'neighbourhood']
    ]
    .drop_duplicates()
    .reset_index(drop=True)
)

# Simulamos auto-increment
nb['neighbourhood_id'] = nb.index + 1

# Reordena columnas
nb = nb[['neighbourhood_id', 'neighbourhood_group', 'neighbourhood']]

nb.to_csv('neighbourhoods.csv', index=False)

In [9]:
df_neighbourhoods = pd.read_csv("neighbourhoods.csv")
df_neighbourhoods.head()

,neighbourhood_id,neighbourhood_group,neighbourhood
0,1,Brooklyn,Kensington
1,2,Manhattan,Midtown
2,3,Manhattan,Harlem
3,4,Brooklyn,Clinton Hill
4,5,Manhattan,East Harlem


In [10]:
df_neighbourhoods['neighbourhood'].nunique()

221

In [11]:
df_neighbourhoods['neighbourhood'].value_counts()

neighbourhood
Kensington             1
Springfield Gardens    1
Norwood                1
Claremont Village      1
Whitestone             1
                      ..
Tribeca                1
Shore Acres            1
Sunset Park            1
Concourse              1
Willowbrook            1
Name: count, Length: 221, dtype: int64

In [12]:
# Mostrar todos los valores que se repiten más de una vez
df_neighbourhoods['neighbourhood'].value_counts()[df_neighbourhoods['neighbourhood'].value_counts() > 1]


Series([], Name: count, dtype: int64)

In [13]:
# Duplicados en neighbourhood_id
print("Duplicados en neighbourhood_id:", df_neighbourhoods['neighbourhood_id'].duplicated().sum())

# Nulos
print(df_neighbourhoods.isnull().sum())

Duplicados en neighbourhood_id: 0
neighbourhood_id       0
neighbourhood_group    0
neighbourhood          0
dtype: int64


### Tabla Listings

In [14]:
# Renombrar id
df_airbnb = df_airbnb.rename(columns={'id': 'listing_id'})

# Hacemos merge para añadir neighbourhood_id
listings = df_airbnb.merge(
    nb,
    on=['neighbourhood_group', 'neighbourhood'],
    how='left'
)

# Nos quedamos con las columnas que queremos
listings = listings[
    ['listing_id', 'name', 'host_id', 'neighbourhood_id',
     'room_type', 'price', 'minimum_nights', 'availability_365']
].copy()

listings.to_csv('listings.csv', index=False)

In [15]:
df_listings = pd.read_csv("listings.csv")
df_listings.head()

,listing_id,name,host_id,neighbourhood_id,room_type,price,minimum_nights,availability_365
0,2539,Clean & quiet apt home by the park,2787,1,Private room,149,1,365
1,2595,Skylit Midtown Castle,2845,2,Entire home/apt,225,1,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,3,Private room,150,3,365
3,3831,Cozy Entire Floor of Brownstone,4869,4,Entire home/apt,89,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,5,Entire home/apt,80,10,0


In [16]:
df_listings['name'].nunique()

47905

In [17]:
print("Duplicados en listing_id:", df_listings['listing_id'].duplicated().sum())
print(df_listings.isnull().sum())

# Validar claves foráneas
print("Hosts no encontrados:", ~df_listings['host_id'].isin(df_hosts['host_id']).all())
print("Neighbourhoods no encontrados:", ~df_listings['neighbourhood_id'].isin(df_neighbourhoods['neighbourhood_id']).all())

Duplicados en listing_id: 0
listing_id           0
name                16
host_id              0
neighbourhood_id     0
room_type            0
price                0
minimum_nights       0
availability_365     0
dtype: int64
Hosts no encontrados: False
Neighbourhoods no encontrados: False


### Tabla Reviews

In [18]:
rev = df_airbnb[
    ['listing_id', 'number_of_reviews', 'last_review', 'reviews_per_month']
].copy()

# Convertir fecha
rev['last_review'] = pd.to_datetime(rev['last_review'])

# Simular AUTO_INCREMENT para review_id
rev = rev.reset_index(drop=True)
rev['review_id'] = rev.index + 1

# Reordenar
rev = rev[
    ['review_id', 'listing_id',
     'number_of_reviews', 'last_review', 'reviews_per_month']
]

rev.to_csv('reviews.csv', index=False)

In [19]:
df_reviews = pd.read_csv("reviews.csv")
df_reviews.head()

,review_id,listing_id,number_of_reviews,last_review,reviews_per_month
0,1,2539,9,2018-10-19,0.21
1,2,2595,45,2019-05-21,0.38
2,3,3647,0,NaN,NaN
3,4,3831,270,2019-07-05,4.64
4,5,5022,9,2018-11-19,0.10


In [20]:
print("Duplicados en review_id (si existe):", df_reviews.get('review_id', pd.Series()).duplicated().sum())
print(df_reviews.isnull().sum())

# Validar claves foráneas
print("Listings no encontrados:", ~df_reviews['listing_id'].isin(listings['listing_id']).all())

# Filas completamente vacías
print("Filas completamente vacías:", df_reviews.isnull().all(axis=1).sum())

Duplicados en review_id (si existe): 0
review_id                0
listing_id               0
number_of_reviews        0
last_review          10052
reviews_per_month    10052
dtype: int64
Listings no encontrados: False
Filas completamente vacías: 0


### Comprobar si se ha importado todo a MYSQL Workbench

In [21]:
print("Hosts:", len(df_hosts))
print("Neighbourhoods:", len(df_neighbourhoods))
print("Listings:", len(df_listings))
print("Reviews:", len(df_reviews))

Hosts: 37457
Neighbourhoods: 221
Listings: 48895
Reviews: 48895
